# ppopgipang-vision-yolo 학습 노트북

이 노트북은 아래처럼 구성된 데이터셋으로 YOLOv8을 학습합니다.

data_root/
  bboxes.json
  classes.txt
  images/
  labels/

가정:
- labels/ 에 YOLO 포맷 라벨이 있습니다 (class x_center y_center width height, 정규화).
- labels/ 가 비어 있고 bboxes.json 이 COCO 형식이면 변환 셀로 라벨을 생성할 수 있습니다.

데이터 경로가 다르면 아래 DATA_ROOT 를 수정하세요.


In [ ]:
!pip install -U ultralytics

from pathlib import Path
import json
import random


## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 파일 확인 및 DATA_ROOT 설정

In [ ]:
DATA_ROOT = Path("/content").resolve()  # notebooks/ 기준 repo 루트
BBOXES_JSON = DATA_ROOT /"bboxes.json"
CLASSES_TXT = DATA_ROOT /"classes.txt"
IMAGES_DIR = DATA_ROOT / "images"
LABELS_DIR = DATA_ROOT / "labels"

print("DATA_ROOT:", DATA_ROOT)
print("images 존재 여부:", IMAGES_DIR.exists(), "labels 존재 여부:", LABELS_DIR.exists())


DATA_ROOT: /content
images 존재 여부: True labels 존재 여부: True


## 출력 결과 경로 지정

In [ ]:
from pathlib import Path
import shutil

# Colab 쪽 결과 경로
RUNS_DIR = Path(DATA_ROOT) / "runs"

# Drive 쪽 저장 경로 (원하는 위치로 수정 가능)
DRIVE_OUT = Path("/content/drive/MyDrive/yolo_results/yolov8_train")

DRIVE_OUT.parent.mkdir(parents=True, exist_ok=True)

## Unknown 클래스 제거

In [ ]:
from pathlib import Path

LABEL_DIR = Path("/content/labels")
UNKNOWN_CLASS_ID = 10  # unknown 클래스 id

removed_boxes = 0
affected_files = 0

for label_file in LABEL_DIR.glob("*.txt"):
    with open(label_file, "r") as f:
        lines = f.readlines()

    new_lines = []
    file_removed = 0

    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue

        class_id = int(parts[0])
        if class_id == UNKNOWN_CLASS_ID:
            file_removed += 1
            removed_boxes += 1
        else:
            new_lines.append(line)

    if file_removed > 0:
        affected_files += 1
        with open(label_file, "w") as f:
            f.writelines(new_lines)

print("✅ unknown 클래스 제거 완료")
print(f"🗑️ 제거된 박스 수: {removed_boxes}")
print(f"📄 영향을 받은 label 파일 수: {affected_files}")


✅ unknown 클래스 제거 완료
🗑️ 제거된 박스 수: 2774
📄 영향을 받은 label 파일 수: 1978


## 클래스 목록 확인

In [ ]:
if CLASSES_TXT.exists():
    class_names = [line.strip() for line in CLASSES_TXT.read_text().splitlines() if line.strip()]
else:
    class_names = []

print("클래스 목록:", class_names)


클래스 목록: ['Chiikawa', 'Hello Kitty', 'Kuromi', 'Mickey Mouse', 'Minions', 'Pikachu', 'Shin-chan', 'Snoopy', 'Stitch', 'Totoro']


In [ ]:
from collections import defaultdict

label_files = list(LABELS_DIR.rglob("*.txt")) if LABELS_DIR.exists() else []
if label_files:
    print(f"라벨 파일이 {len(label_files)}개 있어 변환을 건너뜁니다.")
else:
    if not BBOXES_JSON.exists():
        print("labels/ 가 비어 있고 bboxes.json 도 없습니다. 라벨을 제공하거나 이 셀을 수정하세요.")
    else:
        data = json.loads(BBOXES_JSON.read_text())
        is_coco = isinstance(data, dict) and all(k in data for k in ("images", "annotations", "categories"))
        if not is_coco:
            raise ValueError("bboxes.json 이 COCO 형식이 아닙니다. 스키마에 맞게 이 셀을 수정하세요.")

        coco_categories = {c["id"]: c["name"] for c in data["categories"]}
        if not class_names:
            class_names = [coco_categories[k] for k in sorted(coco_categories)]
            CLASSES_TXT.write_text("\n".join(class_names))
            print("COCO 카테고리로 classes.txt 를 생성했습니다.")

        name_to_idx = {name: i for i, name in enumerate(class_names)}
        missing = [n for n in coco_categories.values() if n not in name_to_idx]
        if missing:
            print("경고: classes.txt 에 없는 카테고리:", missing)

        image_info = {img["id"]: img for img in data["images"]}
        ann_by_image = defaultdict(list)
        for ann in data["annotations"]:
            if ann.get("iscrowd"):
                continue
            ann_by_image[ann["image_id"]].append(ann)

        LABELS_DIR.mkdir(parents=True, exist_ok=True)
        for img_id, info in image_info.items():
            file_name = info.get("file_name")
            if not isinstance(file_name, str):
                continue
            img_rel = Path(file_name)
            if img_rel.parts and img_rel.parts[0] == "images":
                img_rel = Path(*img_rel.parts[1:])

            width = info.get("width")
            height = info.get("height")
            if not width or not height:
                continue

            lines = []
            for ann in ann_by_image.get(img_id, []):
                bbox = ann.get("bbox")
                if not bbox or len(bbox) != 4:
                    continue
                x, y, w, h = bbox
                if w <= 0 or h <= 0:
                    continue

                x_c = (x + w / 2) / width
                y_c = (y + h / 2) / height
                w_n = w / width
                h_n = h / height

                cat_name = coco_categories.get(ann.get("category_id"))
                if cat_name not in name_to_idx:
                    continue
                cls_id = name_to_idx[cat_name]

                lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}")

            label_path = LABELS_DIR / img_rel.with_suffix(".txt")
            label_path.parent.mkdir(parents=True, exist_ok=True)
            label_path.write_text("\n".join(lines))

        print("labels/ 에 YOLO 라벨을 생성했습니다:", LABELS_DIR)


## 학습/검증 분할 및 data.yaml 생성


In [ ]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
train_dir = IMAGES_DIR / "train"
val_dir = IMAGES_DIR / "val"
train_txt = DATA_ROOT / "train.txt"
val_txt = DATA_ROOT / "val.txt"

if train_dir.exists() and val_dir.exists():
    train_spec = train_dir.resolve().as_posix()
    val_spec = val_dir.resolve().as_posix()
elif train_txt.exists() and val_txt.exists():
    train_spec = train_txt.resolve().as_posix()
    val_spec = val_txt.resolve().as_posix()
else:
    image_paths = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in IMAGE_EXTS]
    image_paths = sorted(image_paths)
    if not image_paths:
        raise FileNotFoundError(f"이미지 파일을 찾지 못했습니다: {IMAGES_DIR}")

    random.seed(42)
    random.shuffle(image_paths)
    split = int(len(image_paths) * 0.8)
    train_paths = image_paths[:split]
    val_paths = image_paths[split:]

    train_txt.write_text("\n".join(p.resolve().as_posix() for p in train_paths))
    val_txt.write_text("\n".join(p.resolve().as_posix() for p in val_paths))

    train_spec = train_txt.resolve().as_posix()
    val_spec = val_txt.resolve().as_posix()

print("train 경로:", train_spec)
print("val 경로:", val_spec)


train 경로: /content/train.txt
val 경로: /content/val.txt


In [ ]:
DATA_YAML = DATA_ROOT / "data.yaml"
names_yaml = json.dumps(class_names)
data_yaml_text = "\n".join([
    f"path: {DATA_ROOT.as_posix()}",
    f"train: {train_spec}",
    f"val: {val_spec}",
    f"nc: {len(class_names)}",
    f"names: {names_yaml}",
])
DATA_YAML.write_text(data_yaml_text)
print(DATA_YAML.read_text())


path: /content
train: /content/train.txt
val: /content/val.txt
nc: 10
names: ["Chiikawa", "Hello Kitty", "Kuromi", "Mickey Mouse", "Minions", "Pikachu", "Shin-chan", "Snoopy", "Stitch", "Totoro"]


## YOLOv8 학습


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
results = model.train(
    data="/content/data.yaml",
    epochs=80,
    imgsz=768,
    batch=62,                # A100 기준 실사용 배치
    device=0,                # A100
    amp=True,                # Automatic Mixed Precision
    cache="ram",             # RAM 캐시 (속도 우선)
    # Optimizer (auto → SGD 선택됨)
    optimizer="auto",
    lr0=0.01,
    momentum=0.9,
    weight_decay=5e-4,
    # Data Augmentation (bbox 유지)
    degrees=15,
    translate=0.1,
    scale=0.5,
    shear=1.0,
    fliplr=0.5,
    mosaic=0.3,
    close_mosaic=10,
    # Color Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    # Warmup
    warmup_epochs=3,
    warmup_bias_lr=0.1,
    warmup_momentum=0.8,
    # Training setup
    workers=8,
    val=True,
    patience=100,
    pretrained=True,
    # Output
    project=str(RUNS_DIR),
    name="yolov8m_768_augA100",
)

if DRIVE_OUT.exists():
    shutil.rmtree(DRIVE_OUT)

shutil.copytree(RUNS_DIR, DRIVE_OUT)

print("✅ 학습 결과 Google Drive로 복사 완료")
print("📁 위치:", DRIVE_OUT)

New https://pypi.org/project/ultralytics/8.3.248 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.247 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=30, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_768_aug2, nbs=64, nms=False, opset=Non

## 샘플 이미지 추론


In [ ]:
sample_images = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in IMAGE_EXTS]
if sample_images:
    pred = model.predict(source=str(sample_images[0]), conf=0.25, save=True)
    print("예측 결과 저장 위치:", pred[0].save_dir)



image 1/1 /content/yolo/images/train/32239b0c-9dd2-4e17-a4b2-3defcefa2f4c.jpg: 640x480 4 unknowns, 6.8ms
Speed: 2.0ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/detect/predict
예측 결과 저장 위치: /content/runs/detect/predict


## 클래스 불균형 확인

In [ ]:
from pathlib import Path
from collections import Counter

# DATA_ROOT, LABELS_DIR, IMAGES_DIR, train_paths는 이전 셀에서 정의되었습니다.
# 이 셀에서는 해당 변수들을 직접 사용합니다.

cnt = Counter()

# 훈련 이미지 경로 목록(train_paths)을 사용하여 해당 라벨 파일을 찾습니다.
for image_path in train_paths:
    # 이미지 경로에서 라벨 파일 경로를 구성합니다.
    # 예: /content/images/subdir/img.jpg -> /content/labels/subdir/img.txt
    relative_path_to_image = image_path.relative_to(IMAGES_DIR)
    label_path = LABELS_DIR / relative_path_to_image.with_suffix(".txt")

    if label_path.exists():
        for line in label_path.read_text().splitlines():
            if line.strip():
                # YOLO 포맷: <class_id> <x_center> <y_center> <width> <height>
                parts = line.split()
                if len(parts) > 0:
                    try:
                        cls = int(parts[0]) # 첫 번째 요소가 클래스 ID여야 합니다.
                        cnt[cls] += 1
                    except ValueError:
                        print(f"경고: {label_path} 파일의 '{line}' 라인에서 클래스 ID를 파싱할 수 없습니다. 스킵합니다.")
                else:
                    print(f"경고: {label_path} 파일에 빈 라인 또는 유효하지 않은 라인이 있습니다: '{line}'. 스킵합니다.")
    # else:
    #     print(f"경고: {label_path} 라벨 파일을 찾을 수 없습니다. 해당 이미지의 라벨은 집계되지 않습니다.")

print("총 클래스 등장 수:", len(cnt))
print("top10:", cnt.most_common(20))

if len(cnt) > 0:
    # 클래스가 집계되었을 때만 중앙값을 계산합니다.
    print("클래스당 중앙값 박스 수:", sorted(cnt.values())[len(cnt)//2])
else:
    print("경고: 라벨 파일에서 클래스를 찾을 수 없습니다.")

총 클래스 등장 수: 10
top10: [(9, 603), (1, 543), (7, 515), (6, 387), (0, 369), (5, 339), (4, 270), (2, 258), (3, 242), (8, 203)]
클래스당 중앙값 박스 수: 369
